# 🎄 [Day 3](https://adventofcode.com/2020/day/3)

In [1]:
def parse_forest(inputs):
    return [[int(c == '#')  for c in line] 
            for line in inputs]
    

def slide(forest, slope_right=3, slope_down=1):
    """Compute number of trees on the slide path"""
    x, y = 0, 0
    num_trees = 0
    num_cols = len(forest[0])
    while y < len(forest):
        num_trees += forest[y][x % num_cols]
        x += slope_right
        y += slope_down
    return num_trees
    
    
from functools import reduce
def slide_all_the_way(forest):
    slopes = [(1, 1), (3, 1), (5, 1), (7, 1), (1, 2)]
    return reduce(lambda a, b: a * b,
                  [slide(forest, r, d) for (r, d) in slopes])

In [2]:
with open('inputs/day03.txt', 'r') as f:
    inputs = f.read().splitlines()
forest = parse_forest(inputs)

print(f"We only hit {slide(forest, 3, 1)} trees on the (3, 1) slope !")
print(f"The product of all the trees we crashed into on  these slopes is {slide_all_the_way(forest)}")

We only hit 254 trees on the (3, 1) slope !
The product of all the trees we crashed into on  these slopes is 1666768320


In [3]:
## Adding a bit of animation because sliding is fun
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
tree_img = Image.open('./resources/Items/Fruits/Cherries.png')
tree_img = np.asarray(tree_img.crop((0, 0, 32, 32)), dtype=np.uint8)[:, :, :3]
empty_img = np.zeros((32, 32, 3), dtype=np.uint8)

def mark(img):
    marked_img = np.array(img)
    marked_img[0, :] = 255
    marked_img[:, 0] = 255
    marked_img[-1, :] = 255
    marked_img[:, -1] = 255
    return marked_img

def display_frame(forest, i, j, n, mark_xy=None):
    """Display forest in square [i + n, j + n] + add a frame around position mark_xy"""
    num_cols = len(forest[0])
    imgs = np.concatenate([
        np.concatenate([(mark(tree_img) if mark_xy and xi == mark_xy[0] and xj == mark_xy[1]
                         else tree_img) 
                        if (i + xi < len(forest) and 
                            forest[i + xi][(j + xj) % num_cols])
                        else (mark(empty_img) if mark_xy and xi == mark_xy[0] and xj == mark_xy[1]
                              else empty_img)
                        for xj in range(n)], axis=1)
        for xi in range(n)], axis=0)
    return Image.fromarray(imgs)
    

def slide_with_anim(forest, slope_right=3, slope_down=1, n=5):
    """Slide function, but this time with animation"""
    x, y = 0, 0
    num_trees = 0
    num_cols = len(forest[0])
    num_steps = max(slope_right, slope_down)
    imgs = []
    while y < len(forest):
        # add next frames
        for frame in range(num_steps):
            imgs.append(display_frame(forest, x, y, n, mark_xy=(
                int(frame * slope_down / num_steps), int(frame * slope_right / num_steps))))
        # Update next position
        num_trees += forest[y][x % num_cols]
        x += slope_right
        y += slope_down

    imgs[0].save(fp="day03.gif", format='GIF', append_images=imgs[1:],
                 save_all=True, duration=200, loop=0)
    return num_trees


from IPython.display import HTML
num_trees = slide_with_anim(forest, 3, 2)
widget = HTML(f"<center><br>Hit <b>{num_trees}</b> cherries on slope <b>(3, 2)</b>!<br>"
              "<img src='day03.gif'></center>")
display(widget)